# Write input.md (Choose One Only)

## Comparison

%%writefile input.md
~~~csv
---
table-width: 1.3
---
,pandoc-csv2table,pandoc-placetable,panflute example,my proposal
type,type=simple|multiline|grid|pipe,,,
header,header=yes|no,header=yes|no,header: True|False,header: True|False
caption,caption,caption,title,caption
source,source,file,source,source
aligns,aligns=LRCD,aligns=LRCD,,alignment: LRCD
width,,"widths=""0.5 0.2 0.3""",,"column-width: [0.5, 0.2, 0.3]"
,,inlinemarkdown,,markdown: True|False
,,delimiter,,
,,quotechar,,
,,id (wrapped by div),,
~~~

## Simple Test

%%writefile input.md
~~~csv
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"
~~~

## Full Test (This one goes into the docstring)

In [1]:
%%writefile input.md
~~~csv
---
caption: "*Great* Title"
alignment: LRC
width:
  - 0.1
  - 0.2
  - 0.3
  - 0.4
header: False
markdown: True
...
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"
~~~


Overwriting input.md


## Testing wrong type

%%writefile input.md
~~~csv
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

---
caption: 0.1
header: IDK
markdown: false
...

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"


---
width:
- -0.1
- -0.2
- -0.3
- -0.4
alignment: 0.1
---
~~~


## Testing Zero Table width

In [2]:
%%writefile input.md
~~~csv
,
,
~~~


Overwriting input.md


## Test external CSV @todo

%%writefile input.md
~~~csv
caption: "*Great* Title"
header: False
width: 0.1, 0.2, 0.3, 0.4
alignment: AlignLeft, AlignRight, AlignCenter, AlignDefault
source: ../source/grid.csv
~~~

# Write Filter

In [3]:
%%writefile ../bin/csv-tables.py
#!/usr/bin/env python3

"""
Panflute filter to parse CSV in fenced YAML code blocks

6 metadata keys are recognized:

-   caption: the caption of the table. If omitted, no caption will be inserted.
-   alignment: a string of characters among L,R,C,D, case-insensitive,
    corresponds to Left-aligned, Right-aligned, Center-aligned, Default-aligned respectively.
    e.g. LCRD for a table with 4 columns
    default: DDD...
-   width: a list of relative width corresponding to the width of each columns.
    default: auto calculate from the length of line in a (potentially multiline) cell.
-   table-width: the relative width of the table (comparing to, say, \linewidth).
    default: 1.0
-   header: If true, has a header row. default: true
-   markdown: If CSV table cell contains markdown syntax or not. default: True

When the metadata keys is invalid, the default will be used instead.

e.g.

```markdown
~~~csv
---
caption: "*Great* Title"
alignment: LRC
width:
  - 0.1
  - 0.2
  - 0.3
  - 0.4
header: False
markdown: True
...
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"
~~~
```
"""

import io
import csv
import panflute

def to_bool(x):
    """
    Do nothing if x is boolean,
    return `False` if it is "false" or "no" (case-insensitive).
    """
    if not isinstance(x, bool):
        if str(x).lower() in ("false", "no"):
            x = False
        else:
            x = True
    return x

def get_table_options(options):
    """
    It parses the options output from `panflute.yaml_filter` and
    return it as variables `(caption, alignment, width, table_width, header, markdown)`.
    
    It also check set the followings to default if they are invalid:
    
    - `width` set to `None` when invalid, each element in `width` set to `0` when negative
    - `table_width`: set to `1.0` if invalid or not positive
    - set `header` to `True` if invalid
    - set `markdown` to `True` if invalid
    """
    # get caption
    caption = options.get('caption')
    # get alignment
    alignment = options.get('alignment')
    # get width
    try:
        width = options.get('width')
        width = [(float(x) if float(x) >= 0 else 0) for x in width]
    except (TypeError, ValueError):
        width = None
    # get table_width
    try:
        table_width = options.get('table-width',1.0)
        table_width = float(table_width) if float(table_width) > 0 else 1.0
    except (TypeError, ValueError):
        table_width = 1.0
    # get header
    header = options.get('header',True)
    header = to_bool(header)
    # get markdown
    markdown = options.get('markdown',True)
    markdown = to_bool(markdown)
    return (caption, alignment, width, table_width, header, markdown)

def read_csv(data):
    """
    read csv and return the table in list
    """
    with io.StringIO(data) as f:
        raw_table_list = list(csv.reader(f))
    return raw_table_list

def parse_table_list(raw_table_list, markdown):
    """
    read table in list and return panflute table format
    """
    body = []
    for row in raw_table_list:
        if markdown:
            cells = [panflute.TableCell(*panflute.convert_text(x)) for x in row]
        else:
            cells = [panflute.TableCell(panflute.Plain(panflute.Str(x))) for x in row]
        body.append(panflute.TableRow(*cells))
    return body

def parse_metadata(caption, alignment, width, table_width, raw_table_list):
    """
    `caption` is assumed to contain markdown, as in standard pandoc YAML metadata
    `alignment` string is parsed into pandoc format (AlignDefault, etc.)
    `width` is auto-calculated if not given in YAML
    """
    # parse caption
    if caption != None:
        caption = panflute.convert_text(str(caption))[0].content
    # preparation: get no of columns of the table
    number_of_columns = len(raw_table_list[0])
    ## parse alignment
    if alignment != None:
        alignment = str(alignment)
        parsed_alignment = []
        for i in range(number_of_columns):
            try:
                if alignment[i].lower() == "l":
                    parsed_alignment.append("AlignLeft")
                elif alignment[i].lower() == "c":
                    parsed_alignment.append("AlignCenter")
                elif alignment[i].lower() == "r":
                    parsed_alignment.append("AlignRight")
                else:
                    parsed_alignment.append("AlignDefault")
            except IndexError:
                for i in range(number_of_columns-len(parsed_alignment)):
                    parsed_alignment.append("AlignDefault")
        alignment = parsed_alignment
    # calculate width
    if width == None:
        width_abs = [max([max([len(line) for line in row[i].split("\n")]) for row in raw_table_list]) for i in range(number_of_columns)]
        width_tot = sum(width_abs)
        try:
            width = [width_abs[i]/width_tot*table_width for i in range(number_of_columns)]
        except ZeroDivisionError:
            width = None
    return (caption, alignment, width)

def csv2table(options, data, element, doc):
    # read YAML metadata
    caption, alignment, width, table_width, header, markdown = get_table_options(options)
    # parse csv to list
    raw_table_list = read_csv(data)
    # parse list to panflute table
    body = parse_table_list(raw_table_list, markdown)
    caption, alignment, width = parse_metadata(caption, alignment, width, table_width, raw_table_list)
    # finalize table according to metadata
    header_row = body.pop(0) if header else None
    table = panflute.Table(*body, caption=caption, alignment=alignment, width=width, header=header_row)
    return table

# We'll only run this for CodeBlock elements of class 'csv'
def main(doc=None):
     return panflute.run_filter(panflute.yaml_filter, tag='csv', function=csv2table, strict_yaml=True)

if __name__ == '__main__':
    main()

Overwriting ../bin/csv-tables.py


In [ ]:
!chmod +x ../bin/csv-tables.py

# Output

## Direct output

In [ ]:
!pandoc input.md -t json | ../bin/csv-tables.py

## Native Output

In [ ]:
!pandoc --filter=../bin/csv-tables.py input.md -t native

## Markdown Output

In [ ]:
!pandoc --filter=../bin/csv-tables.py input.md -t markdown